## Alocação de unidades de atendimento móvel do CBMERJ de Campos dos Goytacazes - RJ

In [1]:
# Library
import pandas as pd
from pulp import *
import numpy as np
import googlemaps

from utils.process import stander_text

**Import data**

In [56]:
file = '../data/UNIDADES_DE_SAUDE.xlsx'
pd_dict = pd.read_excel(
    file,
    sheet_name=None # Load sheets in dataframe dictionaries 
)
# Empilhado

df = pd.concat(pd_dict.values(), ignore_index=True) 
df.head()

,ORIGEM,DESTINO,TEMPO (MIN),DISTÂNCIA (KM)
0,DBM 1/5 GUARUS,Alphaville,25 min,"12,3 km"
1,DBM 1/5 GUARUS,Baixa Grande,1h 6 min,"41,4 km"
2,DBM 1/5 GUARUS,Batista Ponto de Coqueiros,1h,"36,2 km"
3,DBM 1/5 GUARUS,Caju,16 min,"8,9 km"
4,DBM 1/5 GUARUS,Campo Limpo,48 min,"25,9 km"


In [57]:
df['EnderecoCliente']=df['DESTINO'] + ', Campos dos Goytacazes, RJ'  

## **Clean data**

Rename columns and reset index

In [58]:
df['DESTINO'] = df['DESTINO'].map(lambda x: x.strip())
df.rename(columns={'TEMPO (MIN)':'TEMPO', 'DISTÂNCIA (KM)':'DISTANCIA' }, inplace=True)

Convert str in numeric  for feature TEMPO

In [59]:
# Foi indentificado uma falta de padronização na coluna TEMPO 
df.query('TEMPO.str.contains("h")').head()

,ORIGEM,DESTINO,TEMPO,DISTANCIA,EnderecoCliente
1,DBM 1/5 GUARUS,Baixa Grande,1h 6 min,"41,4 km","Baixa Grande, Campos dos Goytacazes, RJ"
2,DBM 1/5 GUARUS,Batista Ponto de Coqueiros,1h,"36,2 km","Batista Ponto de Coqueiros, Campos dos Goytaca..."
103,DBM 1/5 GUARUS,São Martinho,1h 9 min,"45,2 km","São Martinho, Campos dos Goytacazes, RJ"
346,HOSP. TRAVESSÃO,Baixa Grande,1h 15 min,"49,4 km","Baixa Grande, Campos dos Goytacazes, RJ"
347,HOSP. TRAVESSÃO,Batista Ponto de Coqueiros,1h 7 min,"44,2 km","Batista Ponto de Coqueiros, Campos dos Goytaca..."


In [60]:
# Padronizar todos os valores para minutos   
df['TEMPO'] = df['TEMPO'].map(stander_text)

In [61]:
df.head()

,ORIGEM,DESTINO,TEMPO,DISTANCIA,EnderecoCliente
0,DBM 1/5 GUARUS,Alphaville,25,"12,3 km","Alphaville, Campos dos Goytacazes, RJ"
1,DBM 1/5 GUARUS,Baixa Grande,66,"41,4 km","Baixa Grande, Campos dos Goytacazes, RJ"
2,DBM 1/5 GUARUS,Batista Ponto de Coqueiros,60,"36,2 km","Batista Ponto de Coqueiros, Campos dos Goytaca..."
3,DBM 1/5 GUARUS,Caju,16,"8,9 km","Caju, Campos dos Goytacazes, RJ"
4,DBM 1/5 GUARUS,Campo Limpo,48,"25,9 km","Campo Limpo, Campos dos Goytacazes, RJ"


## Pivot

In [62]:
df_matrix = df.pivot(columns='ORIGEM', index='DESTINO', values='TEMPO' )
df_matrix.to_excel('matriz_tempo.xlsx')

## Geocode



In [77]:
import os
from dotenv import load_dotenv

load_dotenv()

API_KEY_GOOGLE_MAPS = os.getenv('API_KEY_GOOGLE_MAPS')

In [78]:
def get_lat_log_from_addres(add):
    '''get lat long in graus decimal thourt geocoding
    
    Args
        add (str): Adress      
    return 
        tuple: (lat, long) 
    '''
    gmaps = googlemaps.Client(key=API_KEY_GOOGLE_MAPS)
    res = gmaps.geocode(add)
    return tuple(res[0]['geometry']['location'].values())    

def get_distance_matrix(origin,destination):
    """_summary_

    Args:
        origin (tuple): tuple lat,long
        destination (tuple): tuple lat,long

    Returns:
        int: time i seconds
    """

Carregar lista de endereços

In [79]:
# Endereços 
file = '../data/enderecos.csv'
df_base = pd.read_csv(file,  encoding='ISO-8859-1', delimiter=';')
df_base.head()

,Base,EnderecoBase
0,5 GBM,"Rua Rui Barbosa, 863 - Centro, Campos dos Goyt..."
1,DBM 1/5 GUARUS,"Rua Alci Ferreira - Codim, Campos dos Goytaca..."
2,HOSP. FERREIRA M,"Rua Rocha Leão, 02 - Centro, Campos dos Goytac..."
3,HOSP. G. GUARÚS,"Avenida Senador José Carlos Pereira Pinto, 400..."
4,HOSP. SÃO JOSÉ,Estrada do Açucar sem número - Goytacazes - C...


In [81]:
# Obeter cooderndas geograficas das facilidades
df_base['base_lat_long'] = df_base['EnderecoBase'].map(get_lat_log_from_addres)

,Base,EnderecoBase,base_lat_long
0,5 GBM,"Rua Rui Barbosa, 863 - Centro, Campos dos Goyt...","(-21.7560766, -41.3163947)"
1,DBM 1/5 GUARUS,"Rua Alci Ferreira - Codim, Campos dos Goytaca...","(-21.709383, -41.2875301)"
2,HOSP. FERREIRA M,"Rua Rocha Leão, 02 - Centro, Campos dos Goytac...","(-21.7437715, -41.333381)"
3,HOSP. G. GUARÚS,"Avenida Senador José Carlos Pereira Pinto, 400...","(-21.7413698, -41.3129263)"
4,HOSP. SÃO JOSÉ,Estrada do Açucar sem número - Goytacazes - C...,"(-21.8294589, -41.2949733)"
5,HOSP. TRAVESSÃO,"Avenida Antônio Luiz da Silveira - Centro, Cam...","(-21.6093672, -41.3123019)"
6,PS GUARUS,"Rua André Luís, 111 - Jardim Carioca, Campos ...","(-21.7454022, -41.3228796)"
7,SANDU,"Rua Saldanha Marinho, 59 - Centro, Campos dos ...","(-21.7642258, -41.3148822)"
8,UBS TOCOS,"Avenida José Pereira das Chagas, Tocos (Campos...","(-21.9007729, -41.2831951)"
9,UPA,"Avenida Carlos Alberto Chebabe, 517 - Parque G...","(-21.7258082, -41.3210406)"


In [88]:
df_base.to_csv('../data/base.csv')
df_base

,Base,EnderecoBase,base_lat_long
0,5 GBM,"Rua Rui Barbosa, 863 - Centro, Campos dos Goyt...","(-21.7560766, -41.3163947)"
1,DBM 1/5 GUARUS,"Rua Alci Ferreira - Codim, Campos dos Goytaca...","(-21.709383, -41.2875301)"
2,HOSP. FERREIRA M,"Rua Rocha Leão, 02 - Centro, Campos dos Goytac...","(-21.7437715, -41.333381)"
3,HOSP. G. GUARÚS,"Avenida Senador José Carlos Pereira Pinto, 400...","(-21.7413698, -41.3129263)"
4,HOSP. SÃO JOSÉ,Estrada do Açucar sem número - Goytacazes - C...,"(-21.8294589, -41.2949733)"
5,HOSP. TRAVESSÃO,"Avenida Antônio Luiz da Silveira - Centro, Cam...","(-21.6093672, -41.3123019)"
6,PS GUARUS,"Rua André Luís, 111 - Jardim Carioca, Campos ...","(-21.7454022, -41.3228796)"
7,SANDU,"Rua Saldanha Marinho, 59 - Centro, Campos dos ...","(-21.7642258, -41.3148822)"
8,UBS TOCOS,"Avenida José Pereira das Chagas, Tocos (Campos...","(-21.9007729, -41.2831951)"
9,UPA,"Avenida Carlos Alberto Chebabe, 517 - Parque G...","(-21.7258082, -41.3210406)"


In [82]:
# Obeter cooderndas geograficas das facilidades
df_cliente = df.drop_duplicates(subset=['EnderecoCliente']).copy()

In [84]:
df_cliente['cliente_lat_long'] = df_cliente['EnderecoCliente'].map(get_lat_log_from_addres)

,ORIGEM,DESTINO,TEMPO,DISTANCIA,EnderecoCliente,cliente_lat_long
0,DBM 1/5 GUARUS,Alphaville,25,"12,3 km","Alphaville, Campos dos Goytacazes, RJ","(-21.7711892, -41.3106727)"
1,DBM 1/5 GUARUS,Baixa Grande,66,"41,4 km","Baixa Grande, Campos dos Goytacazes, RJ","(-21.9492435, -41.13520889999999)"
2,DBM 1/5 GUARUS,Batista Ponto de Coqueiros,60,"36,2 km","Batista Ponto de Coqueiros, Campos dos Goytaca...","(-21.9074616, -41.1586879)"
3,DBM 1/5 GUARUS,Caju,16,"8,9 km","Caju, Campos dos Goytacazes, RJ","(-21.7444336, -41.3375268)"
4,DBM 1/5 GUARUS,Campo Limpo,48,"25,9 km","Campo Limpo, Campos dos Goytacazes, RJ","(-21.8595722, -41.2413952)"
...,...,...,...,...,...,...
915,SANDU,Tv. João Seixas,6,"1,8 km","Tv. João Seixas, Campos dos Goytacazes, RJ","(-21.7723261, -41.3237712)"
927,PS GUARUS,Ceasa,13,"6,3 km","Ceasa, Campos dos Goytacazes, RJ","(-21.7178297, -41.3022266)"
1014,PS GUARUS,Penha,23,"9,1 km","Penha , Campos dos Goytacazes, RJ","(-21.7925146, -41.2833674)"
1157,UPH ST EDUARDO,Chave do Paraíso,36,"23,5 km","Chave do Paraíso, Campos dos Goytacazes, RJ","(-21.3949836, -41.4497645)"


In [89]:
df_cliente[['DESTINO', 'EnderecoCliente','cliente_lat_long']].to_csv('../data/clientes.csv')
df_cliente[['DESTINO', 'EnderecoCliente','cliente_lat_long']]

,DESTINO,EnderecoCliente,cliente_lat_long
0,Alphaville,"Alphaville, Campos dos Goytacazes, RJ","(-21.7711892, -41.3106727)"
1,Baixa Grande,"Baixa Grande, Campos dos Goytacazes, RJ","(-21.9492435, -41.13520889999999)"
2,Batista Ponto de Coqueiros,"Batista Ponto de Coqueiros, Campos dos Goytaca...","(-21.9074616, -41.1586879)"
3,Caju,"Caju, Campos dos Goytacazes, RJ","(-21.7444336, -41.3375268)"
4,Campo Limpo,"Campo Limpo, Campos dos Goytacazes, RJ","(-21.8595722, -41.2413952)"
...,...,...,...
915,Tv. João Seixas,"Tv. João Seixas, Campos dos Goytacazes, RJ","(-21.7723261, -41.3237712)"
927,Ceasa,"Ceasa, Campos dos Goytacazes, RJ","(-21.7178297, -41.3022266)"
1014,Penha,"Penha , Campos dos Goytacazes, RJ","(-21.7925146, -41.2833674)"
1157,Chave do Paraíso,"Chave do Paraíso, Campos dos Goytacazes, RJ","(-21.3949836, -41.4497645)"
